# DATA process

In [212]:
import sys
import numpy as np
import pandas as pd
import networkx as nx

graph_1_list = []
graph_1_list_node_1 = []
graph_2_list = []
graph_2_list_node_1 = []
graph_3_list = []
graph_3_list_node_1 = []
graph_4_list = []
graph_5_list = []
graph_6_list = []

with open('graph_1.txt','r') as f:
    for line in f:
        graph_1_list.append(list(line.strip('\n').split(',')))
        
with open('graph_1.txt','r') as f:
    for line in f:
        graph_1_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_1 = [['2', '1'], ['1', '3'], ['1', '4'], ['1', '5'], ['1', '6'], ['3', '1'], ['4', '1'], ['5', '1'], ['6', '1']]
    for edge in append_list_1:
        graph_1_list_node_1.append(edge)
with open('graph_2.txt','r') as f:
    for line in f:
        graph_2_list.append(list(line.strip('\n').split(',')))
        
with open('graph_2.txt','r') as f:
    for line in f:
        graph_2_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_2 = [['2', '1'], ['1', '3'], ['1', '4'], ['1', '5'], ['3', '1'], ['4', '1'], ['5', '1']]
    for edge in append_list_2:
        graph_2_list_node_1.append(edge)
with open('graph_3.txt','r') as f:
    for line in f:
        graph_3_list.append(list(line.strip('\n').split(',')))
with open('graph_3.txt','r') as f:
    for line in f:
        graph_3_list_node_1.append(list(line.strip('\n').split(',')))
    append_list_3 = [['1', '3'], ['1', '4'], ['3', '1'], ['4', '1']]
    for edge in append_list_3:
        graph_3_list_node_1.append(edge)
with open('graph_4.txt','r') as f:
    for line in f:
        graph_4_list.append(list(line.strip('\n').split(',')))
with open('graph_5.txt','r') as f:
    for line in f:
        graph_5_list.append(list(line.strip('\n').split(',')))
with open('graph_6.txt','r') as f:
    for line in f:
        graph_6_list.append(list(line.strip('\n').split(',')))


data_txt = np.loadtxt('IBM_testing_dataset.txt')
IBM_testing_txtDF = pd.DataFrame(data_txt)
IBM_data = []

for IBM_item,customer in enumerate(IBM_testing_txtDF[0]):
    if len(IBM_data) < int(customer):
        IBM_data.append(list())
        IBM_data[int(customer)-1].append(str(IBM_testing_txtDF[2][IBM_item]))
    else:
        IBM_data[int(customer)-1].append(str(IBM_testing_txtDF[2][IBM_item]))

In [213]:
import csv
import pandas as pd
from itertools import combinations
import time

IBM_data =pd.DataFrame(index=range(len(IBM_data)),data=IBM_data)
#Read data from CSV
# data = pd.read_csv('groceries.csv')
data = IBM_data
# data = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
#
#Parameters
minsup = 0.045#float(input("Support-Threshold: "))
minsup = minsup * len(data)
minconf = 0.1#float(input("Confidence-Threshold: "))
# print(len(data))#9835
time_start = time.time()

#Add all data in a list of lists
items = []  
# print(len(data.values[0]))#1列(ROW)有32的物品
for i in range(0, len(data)):  #將所有data中的物品一列一列放到items
     items.append([str(data.values[i, j]) for j in range(0, len(data.values[0]))])

#Creating a list of dictionaries
count = [dict() for x in range(len(data.values[0]) + 1)]
# print(items)
# #Count support for each individual items
s = []
for i in items:
    for j in i:      #讀取所有items
        s.append(j)
for i in s:                      #創字典 放在count[1]
    #If item is present in dictionary, increment its count by 1
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #If item is not present in dictionary, set its count to 1
    else:
        count[1][i] = 1     #{'i':1}

# #Remove infrequent and empty items #去除小於minsup,nan
for i in count[1].copy():
    if(count[1][i] < minsup):
        count[1].pop(i)
# count[1].pop('nan') #去除以key = 'NAN'產生的值
count[1].pop('None') #去除以key = 'NAN'產生的值

# #Storing transactions as lists without infrequent items
a = list(count[1]) #列出所有符合的KEY
item = [list() for i in range(len(data))] #創造len(data) = 9835個list 
c = 0 
for i in range(0,len(items)): ##將所有data中的物品一列一列放到items
    for j in range(len(items[i])):
#         print(items[i][j])
        if(a.__contains__(items[i][j]) != 0): #確認物品是否存在
            item[i].append(items[i][j]) #將物品分成[['a', 'b', 'c'],[]]與itmes不同的地方是裡練已經沒有NAN了

#Function to sort list to support
def sort(a): #a = 所有符合的KEY的list
    for i in range(len(a) - 1):
        for j in range(len(a) - i - 1):
            if(count[1][a[j]] < count[1][a[j + 1]]): #如果下一個比上一個大則互換位子
                a[j],a[j + 1] = a[j + 1],a[j]
                
#Call function to sort all transactions in descending order of their support
for i in range(0,len(data)): #9835
    if(len(item[i]) > 1): #如果不只一個物件就排序他
        sort(item[i])

#Tree class for FP-Tree
class tree: #設計樹根，分支
    def __init__(self, name, sup, parent):
        self.name = name
        self.sup = sup
        self.nodeLink = None
        self.parent = parent
        self.children = []

#Function to check if the node is present is a child of the current node
def ispresent(node,name):    #確認是否有children
    f = -1               
    for i in node.children:
        f += 1
        if(i.name == name):
            return f
    return -1

#HeaderTable which stores the reference of last/first occurence of an item. Used as a linked list to generate candidate trees 
lastocc = count[1].copy() #符合sup的字典
for i in lastocc: #lastocc字典中所有值皆為NONE
    lastocc[i] = None
#Function to create FP-tree
root = tree("root", -1, None)
z = 0
for i in item: #item為沒有NAN的items
    current = root
    for j in range(len(i)): #len(i)9835
        if(ispresent(current,i[j]) >= 0): #第一次通常不符合此情形，直接執行else，看有沒有children
            current = current.children[ispresent(current, i[j])]
            current.sup = current.sup + 1
        else:
            child=tree(i[j], 1, current)    #創建i[j]子根
            current.children.append(child) #將子根放入children =[] 
            t = current #t 記錄柱上一層的tree1
            current = current.children[ispresent(current, i[j])] #現在current.children裡有子根，所以self.name = i[j]如下示意，
            #ispresent(current,i[j])回傳 0 ，現在的current為tree2
            current.parent = t #紀錄上一層tree1
            if(lastocc[current.name] == None):
                lastocc[current.name] = current #創建{'i[j]': tree2-1()} 
            else:#創出第一層並列樹枝
                current.nodeLink = lastocc[current.name] #將tree2-2連結到tree4
                lastocc[current.name] = current #CURRENT = TREE2-2 {'i[j]': tree2-2}
            
# class tree1:
#     def __init__(self, name, sup, parent):
#         self.name = 'root'
#         self.sup = -1
#         self.nodeLink = None
#         self.parent = none
#         self.children = [
#   class tree2-1:
#     def __init__(self, name, sup, parent):      [['A','B','C'],['C','A']]
#         self.name = 'A'
#         self.sup = 1
#         self.nodeLink = NONE
#         self.parent = parent
#         self.children = [
#                                     class tree3:
#                                         def __init__(self, name, sup, parent):
#                                             self.name = 'B'
#                                             self.sup = 1
#                                             self.nodeLink = None
#                                             self.parent = parent
#                                             self.children = [
#                                                                 class tree4:
#                                                                     def __init__(self, name, sup, parent):
#                                                                         self.name = 'B'
#                                                                         self.sup = 1
#                                                                         self.nodeLink = None
#                                                                         self.parent = parent
#                                                                         self.children = []
#                                                                 ]
#         ],
#   class tree2-2:
#     def __init__(self, name, sup, parent):
#         self.name = 'C'
#         self.sup = sup
#         self.nodeLink = TREE 4
#         self.parent = TREE1
#         self.children = []
# ]



#Function to get frequent itemsets with suffix 'node' and length n
def singlepath(node, n):
    c = 0
    sup = node.sup
    path = []
    pathname = []
    current = node
    
    #Get the path from current node to root
    while(current.parent != None): #查看目current是否為'root'，如果是則無parent，重複做直道回到最上層('root')
        path.append(current)     #將current內容紀錄(tree)
        pathname.append(current.name)      #('c')
        current = current.parent  #往上一層移動
    path.remove(node) #除了自己以外的路線
    pathname.remove(node.name) 
    candidatepath = []
    temp_candidatepath = []
   
    #Generate combinations of length n in the path
    a = (list(combinations(pathname, n))) #列出除了自己以外的長度為n的組合
    for j in a:
        temp_candidatepath.append(tuple(sorted(j)))   #以tuple形式存在temp_candidatapath裡
    #Append the suffix 'node.name' to the above paths
    for j in temp_candidatepath:
        j = list(j) #再次變回list
        j.append(node.name) #將原本剔除的當前TREE名加回去 變成包含當前的組合，就是在做freqent pattern
        candidatepath.append(sorted(j)) #在加回去變成 [['a','c'],['b','c']]
    #Update counts of the generated itemsets
    for j in candidatepath:
        j = tuple(j) #j = ('a','c')
        if j in count[n + 1]: #count[1]為符合minsup的字典，n至少會從1開始 
            count[n + 1][j] = count[n + 1][j] + sup  # 將之前已創好的value值，加上當前tree所有的sup
        else:
            count[n + 1][j] = sup #創建新字典,此sup為當前tree存有的sup
            
    #Iterating in the candidate tree recursively 
    if(node.nodeLink != None): #查看是否存在分支
        node=node.nodeLink
        singlepath(node, i) #i為組合長度
    
#Check if itemset is frequent
def frequent(n): #算出現次數
    f=0
    for i in count[n]:
        if(count[n][i] >= minsup):
            f = 1
    if(f == 1):
        return 1
    else:
        return 0

#Call singlepath function for all frequent nodes
for i in range(1, len(data.values[0]) + 1): #1列(ROW)有32的物品, +1是確保有32為一組的組合  #!
    if(frequent(i) == 1):
        for j in lastocc: #開始製造組合，將結果放到count[]裡，count唯有32空字典的list
            singlepath(lastocc[j], i)

#Remove infrequent itemsets
for z in range(len(data.values[0]) + 1):            
    for i in count[z].copy():
            if(count[z][i] < minsup): #查看所有的dict裡面的東西，是否符合minsup
                count[z].pop(i) #去掉不符合的

frequent_pattern = []
for items in count:
    for want in items:
        frequent_pattern.append(want)
                
#Get 'q', the length of the longest itemset
i=1 #設2是因為從第二筆的字典，才開始有東西
while(len(count[i]) != 0): #從這邊就能知道最大符合長度是多長，因為不符合的都被刪掉了，所以count最大的子矩陣
    i = i + 1
q = i - 1

#Find maximal and closed itemsets
maximal = []
closed = []
for i in range(1, q):
    for j in count[i]:
        fm = 0
        fc = 0
        for k in count[i + 1]:
            a = set(list([j]))
            b = set(list(k))
            #Set is maximal if no immediate superset is frequent
            if(a.intersection(b) == a): #如果兩者重疊則maximal
                fm = 1 
                #Set is closed if none of its immediate supersets have equal support
                if(count[i][j] == count[i + 1][k]): #出現次數相同
                    fc = 1
        if(fm == 0):
            maximal.append(j)
        if(fc == 0):
            closed.append(j)
#All sets at the top of the tree are automatically maximal and closed
for i in count[q]:
    maximal.append(i)
    closed.append(i)
    
# print("frequent_pattern")
# print('number of frequent pattern',len(frequent_pattern))
# print(frequent_pattern)
 
#Find Association Rules 
print("Association_Rules")
all_frequent_pattern = {}
for len_FP in count:
    all_frequent_pattern.update(len_FP)

IBM_list = []
def rulegenerator(fitems): #fitmes = 最終dic
    '''
    Generates association rules from the frequent itemsets
    '''
    counter = 0
    
    for itemset in fitems.keys():
        if isinstance(itemset, str): #只有單個得我不要，我要tuple形式('a','b')
            continue
        length = len(itemset)#tuple裡面有幾項
        union_support = fitems[tuple(itemset)]#原本itemset為LIST形式，必須使用tuple(才可以變成key ，看兩個以上的品項的出現次數
        for i in range(1, length):
            lefts = map(list, combinations(itemset, i)) #['mineral water'], ['pancakes']
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]#單品項出現次數 
                        conf = union_support / leftcount
                y_list = []
                x_list = []
                if conf >= minconf:
                    right = list(itemset[:])
                    sub_list = []
                    for e in left:                          #去除被分配在左邊的品項
                        right.remove(e)     
#                     fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')
                    
                    
#                     print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
#                     for y in left:
#                         y_list.append(int(float(y)))
#                     for x in right:
#                         x_list.append(int(float(x)))
                    for y in left:
                        y_list.append(y)
                    for x in right:
                        x_list.append(x)
                    sub_list.append(y_list)
                    sub_list.append(x_list)
                    IBM_list.append(sub_list)
                    counter += 1
#                     fo.close()
    print(counter, "rules generated")

rulegenerator(all_frequent_pattern)






time_end = time.time()
print('FP_Growth_spent_time', time_end - time_start)
# print( time_end - time_start)
# print(len(ant))

Association_Rules
1088 rules generated
FP_Growth_spent_time 0.3966660499572754


# graph_1

In [3]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_1_list
Damping_factor = 0.15

def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):#取零只是我想找len隨意設的
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) + (1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'D = {Damping_factor}')
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_1_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

22
9.209645584854625e-05
6
[0.03480556 0.07599656 0.12474286 0.18241334 0.25064252 0.33139916]
time cost 0.002385854721069336


# graph_1_power_node_1

In [4]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_1_list_node_1
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):#取零只是我想找len隨意設的
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_1_PageRank_power_node_1_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

15
6.244088680754101e-05
6
[0.35987982 0.08617426 0.1228026  0.1383662  0.14498351 0.14779362]
time cost 0.002293825149536133


# graph_2

In [5]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = graph_2_list
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_2_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

1
0.0
5
[0.2 0.2 0.2 0.2 0.2]
time cost 0.004675149917602539


# graph_2_power_node_1

In [6]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_2_list_node_1
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):#取零只是我想找len隨意設的
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_2_PageRank_power_node_1_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

13
6.558517908100692e-05
5
[0.37426656 0.10952468 0.15607922 0.17585877 0.18427078]
time cost 0.003728151321411133


# graph_3

In [7]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_3_list
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_3_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

8
7.275616239243998e-05
4
[0.19908329 0.30091671 0.30091671 0.19908329]
time cost 0.001787424087524414


# graph_3_power_node_1

In [8]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_3_list_node_1
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):#取零只是我想找len隨意設的
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_3_PageRank_power_node_1_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

15
9.974359202091776e-05
4
[0.29522179 0.20477821 0.29522179 0.20477821]
time cost 0.008326292037963867


# graph_4

In [9]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = graph_4_list
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_4_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

11
4.461390240563523e-05
7
[0.19846218 0.17338185 0.17220318 0.13250982 0.16827854 0.07305883
 0.08210561]
time cost 0.0044820308685302734


# graph_5

In [10]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = graph_5_list
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_5_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

36
9.336569216016359e-05
469
[0.00021951 0.00021951 0.00021951 0.00021951 0.00021951 0.00026219
 0.00026219 0.00026219 0.00026219 0.00026219 0.00026219 0.00026219
 0.00026219 0.00026219 0.00026219 0.00027383 0.00027211 0.00038798
 0.00028737 0.00037416 0.02233096 0.02233096 0.00037416 0.03196575
 0.00033366 0.00027049 0.00028041 0.00027211 0.00027211 0.00029567
 0.00028112 0.00027211 0.00031344 0.00029225 0.02226695 0.00040125
 0.00027934 0.00030766 0.00037416 0.00186319 0.00027211 0.00037416
 0.04954971 0.07201541 0.00027211 0.00075966 0.00035621 0.00036189
 0.00027383 0.00036749 0.00027383 0.00027211 0.00027211 0.00040125
 0.00054908 0.00027934 0.00027418 0.00036873 0.00027404 0.00062308
 0.00813812 0.00027418 0.00034899 0.00027211 0.00033317 0.00027404
 0.00037416 0.00027383 0.00027211 0.01461162 0.00042214 0.00029225
 0.00027418 0.00027211 0.00030766 0.00027211 0.00027211 0.00030766
 0.00029121 0.00027404 0.00054908 0.00037416 0.00037416 0.00040244
 0.00033366 0.00027404 0.00027404

# graph_6

In [11]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = graph_6_list
Damping_factor = 0.15
def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"graph_6_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

149
9.761824045171674e-05
1228
[3.14080521e-05 3.89500000e-05 2.66436158e-04 ... 7.69410170e-04
 3.32547444e-04 2.78054235e-04]
time cost 105.29113173484802


# IBM

In [214]:
node_list = {}
for idx, x in enumerate(all_frequent_pattern.keys()):
    node_list[x] = idx

In [216]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = IBM_list
Damping_factor = 0.15


# def find_dim(wnat_list):
#     dim = np.max(want_list)
#     return dim

# dim = int(float(find_dim(want_list)[0]))
dim = len(node_list)
adj_matrix_hub = np.zeros((dim, dim))
last_pagerank = np.ones(dim)/dim

# 建hub_matrix
for idx,transaction in enumerate(want_list):
    if (len(transaction[0]) != 1):
        x_location = node_list[tuple(transaction[0])]
    elif len(transaction[1]) != 1:
#         print(transaction, len(transaction[0]), len(transaction[1]))
        y_location = node_list[tuple(transaction[1])]
    else:
        x_location = node_list[transaction[0][0]]
        y_location = node_list[transaction[1][0]]
    
    adj_matrix_hub[y_location][x_location] += 1



        
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
        np.savetxt(f"IBM_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

9
4.5179538432954264e-05
171
[0.00479122 0.00583268 0.00583268 0.00479122 0.00479122 0.00537928
 0.00537928 0.00537928 0.00537928 0.00537928 0.00537928 0.00329418
 0.00357251 0.00295115 0.00384807 0.00384807 0.00384807 0.00448954
 0.00329418 0.00257913 0.00340078 0.00340078 0.00287893 0.00287893
 0.00447731 0.00656753 0.00504538 0.00504538 0.00504538 0.00702989
 0.00257913 0.00540594 0.00540594 0.00400406 0.00446947 0.00993803
 0.01023024 0.00847894 0.00960319 0.00859147 0.01068369 0.01836931
 0.0060343  0.01065305 0.00673568 0.02149602 0.0123587  0.01326158
 0.01484455 0.02478219 0.00804438 0.01065305 0.01906937 0.02528434
 0.02557363 0.02249163 0.00287893 0.00257913 0.00487766 0.00321356
 0.00509426 0.0035871  0.0035871  0.00600868 0.00393788 0.00287893
 0.00287893 0.00287893 0.00441129 0.00257913 0.0035871  0.0035871
 0.00321356 0.00754109 0.00462198 0.00540594 0.00446947 0.00540594
 0.00400406 0.0063919  0.00658886 0.00798595 0.00994753 0.0117904
 0.01029018 0.003626   0.0035871  0

# static_test

In [239]:
print(f"D = {Damping_factor}")
print(want_threshold_error)
# print(new_pagerank)
print(times_graph_2)
print(f'time cost {spend_time}')

D = 0.15
0.0001
11
time cost 0.001951456069946289


In [238]:
import numpy as np
import time

start_time = time.time()

want_threshold_error = 0.0001
want_list = graph_3_list
Damping_factor = 0.15

def find_dim(wnat_list):
    all_number_list = []
    for x in want_list:
        for num in x:
            all_number_list.append(int(num))
    dim = max(all_number_list)
    return dim

dim = find_dim(want_list)
adj_matrix_hub = np.zeros((dim, dim))


last_pagerank = np.ones(dim)/dim
# 建hub_matrix
for idx, hub_point in enumerate(want_list):
    y_location = int(hub_point[0]) - 1
    x_location = int(hub_point[1]) - 1
    adj_matrix_hub[y_location][x_location] += 1
    
    
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):#取零只是我想找len隨意設的
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                current_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) + (1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        
        print(f"D = {Damping_factor}")
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
#         np.savetxt(f"graph_1_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

D = 0.15
11
8.171056820782674e-05
4
[0.1754325 0.3245675 0.3245675 0.1754325]
time cost 0.001951456069946289


In [221]:
import numpy as np
import time

start_time = time.time()
want_threshold_error = 0.0001
want_list = IBM_list
Damping_factor = 0.1


# def find_dim(wnat_list):
#     dim = np.max(want_list)
#     return dim

# dim = int(float(find_dim(want_list)[0]))
dim = len(node_list)
adj_matrix_hub = np.zeros((dim, dim))
last_pagerank = np.ones(dim)/dim

# 建hub_matrix
for idx,transaction in enumerate(want_list):
    if (len(transaction[0]) != 1):
        x_location = node_list[tuple(transaction[0])]
    elif len(transaction[1]) != 1:
#         print(transaction, len(transaction[0]), len(transaction[1]))
        y_location = node_list[tuple(transaction[1])]
    else:
        x_location = node_list[transaction[0][0]]
        y_location = node_list[transaction[1][0]]
    
    adj_matrix_hub[y_location][x_location] += 1



        
adj_matrix_authority = adj_matrix_hub.transpose()
times_graph_2 = 0    
while True:    
    times_graph_2 += 1
    new_pagerank = []
    


    for i in range(len(adj_matrix_authority)): #i代表當前再算那個node
        
        
        parent_idx =[]
        
        #找parent outlink
        for idx in range(len(adj_matrix_authority[0])):
            if adj_matrix_authority[i][idx] != 0:
                parent_idx.append(idx)
        if len(parent_idx) == 0:
            sum_parent_pagelink = 0
        else:
            sum_parent_pagelink = 0
            for idx in parent_idx:
                out_links = 0
                parent_outlinks_list = adj_matrix_hub[idx]
                for out_link in current_outlinks_list:
                    if out_link != 0:
                        out_links += 1
                sum_parent_pagelink += last_pagerank[idx]/out_links
        page_rank_damping = (Damping_factor/dim) +(1-Damping_factor) * sum_parent_pagelink
        new_pagerank.append(page_rank_damping)
        
    new_pagerank_value = sum(np.absolute(new_pagerank))
    last_pagerank_value = sum(np.absolute(last_pagerank))
    new_pagerank = new_pagerank/new_pagerank_value
    last_pagerank_nomalize = last_pagerank/last_pagerank_value
    dif_pagerank = sum(np.absolute(new_pagerank - last_pagerank_nomalize)) 
    
    if dif_pagerank < want_threshold_error:
        print(f'{times_graph_2}')
        print(dif_pagerank)
        print(len(new_pagerank))
        print(new_pagerank)
        end_time = time.time()
        spend_time = end_time - start_time 
        print(f'time cost {spend_time}')
#         np.savetxt(f"IBM_PageRank_D={Damping_factor}.txt", new_pagerank, newline = ' ', fmt = '%10.5f')
        break
    
    last_pagerank = new_pagerank

11
8.319557731933675e-05
171
[0.00461867 0.00586995 0.00586995 0.00461867 0.00461867 0.00534313
 0.00534313 0.00534313 0.00534313 0.00534313 0.00534313 0.00268039
 0.00294304 0.00234178 0.00330108 0.00330108 0.00330108 0.00393449
 0.00268039 0.00197    0.00282375 0.00282375 0.00225484 0.00225484
 0.00431648 0.00684422 0.00504412 0.00504412 0.00504412 0.0074112
 0.00197    0.00485683 0.00485683 0.0033812  0.0038701  0.01077752
 0.01139085 0.00885989 0.01042105 0.0090771  0.01214023 0.02330287
 0.0055591  0.01081986 0.00636291 0.02800603 0.01453482 0.01531782
 0.01802669 0.0324748  0.00785275 0.01081986 0.02383426 0.03387743
 0.03474033 0.02892242 0.00225484 0.00197    0.00440333 0.00258088
 0.0045721  0.00295406 0.00295406 0.00545094 0.00332596 0.00225484
 0.00225484 0.00225484 0.00396918 0.00197    0.00295406 0.00295406
 0.00258088 0.00766217 0.00431558 0.00485683 0.0038701  0.00485683
 0.0033812  0.00609588 0.00655957 0.00822234 0.01049402 0.01247933
 0.01100171 0.0030012  0.00295406 

In [222]:
len(new_pagerank)

171

In [15]:
for x in range(3):
    t = 0
    while True:
        t +=1
        break

In [16]:
t

1